In [1]:
from dataclasses import dataclass
from copy import deepcopy


In [2]:

@dataclass(frozen=True)
class Game_State:
    board_positions: list[list]
    moves_left: int
    to_go_next: str
    
    def print_board(self):
        for row in self.board_positions:
            print("|".join(["." if e == "" else e for e in row]))
    
    def is_game_over(self):
        """Returns False, none for still going and True, 1/0/-1 for over.
        1 is user wins, 0 is draw, -1 is user looses"""
        def gen_triplets():
            # yield rows
            yield from self.board_positions
            # yield columns
            for col in range(3):
                yield [self.board_positions[row][col] for row in range(3)]
            # yield diagonals
            yield [
                self.board_positions[0][0],
                self.board_positions[1][1],
                self.board_positions[2][2],
            ]
            yield [
                self.board_positions[2][0],
                self.board_positions[1][1],
                self.board_positions[0][2],
            ]
        if ["X"]*3 in gen_triplets():
            return True, 1
        if ["O"]*3 in gen_triplets():
            return True, -1
        # note so long as user takes all odd turns 9,7,...3,1 they will have last go so redundant
        if self.moves_left == 0:
            return True, 0
        return False, None
        

    def gen_child_game_states(self):
        "precondition of game not being over"
        for i, row in enumerate(self.board_positions):
            for j, square in enumerate(row):
                if square == "":
                    new_board_positions = deepcopy(self.board_positions)
                    new_board_positions[i][j] = self.to_go_next
                    
                    yield Game_State(
                        board_positions=new_board_positions,
                        moves_left=self.moves_left-1,
                        to_go_next="X" if self.to_go_next == "O" else "O"
                    )


In [28]:
# original function
def british_museum_minimax(game_state: Game_State, maximizing_player: bool, alpha, beta):
    # computer want 0 or -1 so minimizer
    over, outcome = game_state.is_game_over()
    if over:
        return outcome, game_state

    if maximizing_player:
        max_evaluation = -100
        for child in game_state.gen_child_game_states():
            evaluation = british_museum_minimax(
                game_state=child,
                maximizing_player=not maximizing_player,
                alpha=alpha,
                beta=beta,
            )[0]
            if evaluation > max_evaluation:
                max_evaluation = evaluation
                best_child = child
            # max_evaluation = max(max_evaluation, evaluation)
            alpha = max(alpha, evaluation)
            if beta <= alpha:
                break
        return max_evaluation, best_child
    else:
        min_evaluation = 100
        for child in game_state.gen_child_game_states():
            evaluation = british_museum_minimax(
                game_state=child,
                maximizing_player=not maximizing_player,
                alpha=alpha,
                beta=beta,
            )[0]
            if evaluation < min_evaluation:
                min_evaluation = evaluation
                best_child = child
            # min_evaluation = min(min_evaluation, evaluation)
            beta = min(alpha, evaluation)
            if beta <= alpha:
                break
        return min_evaluation, best_child


In [29]:
score, best_child = british_museum_minimax(
    game_state=Game_State(
        moves_left=6,
        to_go_next="O",
        board_positions=[
            ["O", "X", ""],
            ["", "X", ""],
            ["", "", ""],
        ]
    ),
    maximizing_player=False,
    alpha=-100,
    beta=+100
)

In [30]:
score

1

In [31]:
best_child.print_board()

O|X|O
.|X|.
.|.|.


# This shows that the current minimax algorithm is flawed as it thinks that the best it can possible do it a loss and it has picked a move that is a loss if the user acts optimally

In [32]:
score, best_child = british_museum_minimax(
    game_state=Game_State(
        moves_left=2,
        to_go_next="O",
        board_positions=[
            ["O", "X", "X"],
            ["O", "X", "O"],
            ["", "", "X"],
        ]
    ),
    maximizing_player=False,
    alpha=-100,
    beta=+100
)


In [33]:
score

-1

In [34]:
best_child.print_board()

O|X|X
O|X|O
O|.|X


Here we see that it is able to recognize that it can win which is better than the only other option which is a draw

Perhaps the issue is with the game over function incorrectly determining who wins

In [35]:
Game_State(
    moves_left=0,
    to_go_next="O",
    board_positions=[
        ["O", "X", "X"],
        ["O", "X", "O"],
        ["X", "O", "X"],
    ]
).is_game_over()


(True, 1)

Thats correct

In [36]:
Game_State(
    moves_left=0,
    to_go_next="O",
    board_positions=[
        ["O", "X", "X"],
        ["X", "O", "O"],
        ["X", "O", "X"],
    ]
).is_game_over()


(True, 0)

thats also correct

In [37]:
Game_State(
    moves_left=0,
    to_go_next="O",
    board_positions=[
        ["X", "O", "O"],
        ["X", "O", "X"],
        ["O", "X", "O"],
    ]
).is_game_over()


(True, -1)

this is also correct

I will now try to make a tree to better visualize what is happening

In [38]:
@dataclass
class Tree:
    root_node_text: str
    sub_trees: list
    
    def print_tree(self, indent=0):
        # print(f"printing tree {self.root_node_text} with indent {indent}")
        print("|" + "---" *indent +f"-->{self.root_node_text}")
        for sub_tree in self.sub_trees:
            sub_tree.print_tree(indent=indent+1)

In [39]:
Tree("A",[
    Tree("B",[
        Tree("E", [])
    ]),
    Tree("C",[
        Tree("D", [
            Tree("F", []),
            Tree("G", []),
        ])
    ]),
]).print_tree()

|-->A
|----->B
|-------->E
|----->C
|-------->D
|----------->F
|----------->G


that isn't a great tree

maybe I could just add a load of print statements 

In [3]:
def print_dec(func):
    func_name = func.__name__
    def wrapper(*args, **kwargs):
        print(f"started:  {func_name}(args={args}, kwargs={kwargs})")
        result = func(*args, **kwargs)
        print(f"finished:  {func_name}(args={args}, kwargs={kwargs})  return value:   {result}")
        return result
    
    wrapper.__name__ = func_name
    return wrapper

In [4]:
# new transparent version
@print_dec
def british_museum_minimax(game_state: Game_State, maximizing_player: bool, alpha, beta):
    # computer want 0 or -1 so minimizer
    over, outcome = game_state.is_game_over()
    if over:
        return outcome, game_state

    if maximizing_player:
        max_evaluation = -100
        for child in game_state.gen_child_game_states():
            evaluation = british_museum_minimax(
                game_state=child,
                maximizing_player=not maximizing_player,
                alpha=alpha,
                beta=beta,
            )[0]
            if evaluation > max_evaluation:
                max_evaluation = evaluation
                best_child = child
            # max_evaluation = max(max_evaluation, evaluation)
            alpha = max(alpha, evaluation)
            if beta <= alpha:
                break
        return max_evaluation, best_child
    else:
        min_evaluation = 100
        for child in game_state.gen_child_game_states():
            evaluation = british_museum_minimax(
                game_state=child,
                maximizing_player=not maximizing_player,
                alpha=alpha,
                beta=beta,
            )[0]
            if evaluation < min_evaluation:
                min_evaluation = evaluation
                best_child = child
            # min_evaluation = min(min_evaluation, evaluation)
            beta = min(alpha, evaluation)
            if beta <= alpha:
                break
        return min_evaluation, best_child


In [5]:
score, best_child = british_museum_minimax(
    game_state=Game_State(
        moves_left=2,
        to_go_next="O",
        board_positions=[
            ["O", "X", "X"],
            ["O", "X", "O"],
            ["", "", "X"],
        ]
    ),
    maximizing_player=False,
    alpha=-100,
    beta=+100
)


started:  british_museum_minimax(args=(), kwargs={'game_state': Game_State(board_positions=[['O', 'X', 'X'], ['O', 'X', 'O'], ['', '', 'X']], moves_left=2, to_go_next='O'), 'maximizing_player': False, 'alpha': -100, 'beta': 100})
started:  british_museum_minimax(args=(), kwargs={'game_state': Game_State(board_positions=[['O', 'X', 'X'], ['O', 'X', 'O'], ['O', '', 'X']], moves_left=1, to_go_next='X'), 'maximizing_player': True, 'alpha': -100, 'beta': 100})
finished:  british_museum_minimax(args=(), kwargs={'game_state': Game_State(board_positions=[['O', 'X', 'X'], ['O', 'X', 'O'], ['O', '', 'X']], moves_left=1, to_go_next='X'), 'maximizing_player': True, 'alpha': -100, 'beta': 100})  return value:   (-1, Game_State(board_positions=[['O', 'X', 'X'], ['O', 'X', 'O'], ['O', '', 'X']], moves_left=1, to_go_next='X'))
finished:  british_museum_minimax(args=(), kwargs={'game_state': Game_State(board_positions=[['O', 'X', 'X'], ['O', 'X', 'O'], ['', '', 'X']], moves_left=2, to_go_next='O'), 'ma

In [43]:
best_child.print_board()

O|X|X
O|X|O
O|.|X


In [44]:
score

-1

that makes sense, lets try to do this for one that doesn't work 

In [45]:
score, best_child = british_museum_minimax(
    game_state=Game_State(
        moves_left=6,
        to_go_next="O",
        board_positions=[
            ["O", "X", ""],
            ["", "X", ""],
            ["", "", ""],
        ]
    ),
    maximizing_player=False,
    alpha=-100,
    beta=+100
)

best_child.print_board()
print(score)

started:  british_museum_minimax(args=(), kwargs={'game_state': Game_State(board_positions=[['O', 'X', ''], ['', 'X', ''], ['', '', '']], moves_left=6, to_go_next='O'), 'maximizing_player': False, 'alpha': -100, 'beta': 100})
started:  british_museum_minimax(args=(), kwargs={'game_state': Game_State(board_positions=[['O', 'X', 'O'], ['', 'X', ''], ['', '', '']], moves_left=5, to_go_next='X'), 'maximizing_player': True, 'alpha': -100, 'beta': 100})
started:  british_museum_minimax(args=(), kwargs={'game_state': Game_State(board_positions=[['O', 'X', 'O'], ['X', 'X', ''], ['', '', '']], moves_left=4, to_go_next='O'), 'maximizing_player': False, 'alpha': -100, 'beta': 100})
started:  british_museum_minimax(args=(), kwargs={'game_state': Game_State(board_positions=[['O', 'X', 'O'], ['X', 'X', 'O'], ['', '', '']], moves_left=3, to_go_next='X'), 'maximizing_player': True, 'alpha': -100, 'beta': 100})
started:  british_museum_minimax(args=(), kwargs={'game_state': Game_State(board_positions=[

well that didn't help